- prob 1: remove constant column from securities VVV
- prob 2: create a staightified cv/confirm VVV
- prob 3: tune rf VVV
- prob 4: pca (normalize)? VVV
- prob 5: what lasso decides? VVV

In [1]:
import numpy as np
import pandas as pd
import time
import random
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
adjusted = pd.read_csv('./input/prices-split-adjusted.csv')
fundamentals = pd.read_csv('./input/fundamentals.csv', index_col=0)
securities = pd.read_csv('./input/securities.csv')

In [3]:
to_be_removed = ['Period Ending','Cash Ratio', 'Quick Ratio', 'Current Ratio', 
                 'Estimated Shares Outstanding', 'Earnings Per Share', 'For Year']

In [4]:
tmp = fundamentals[filter(lambda col: col not in to_be_removed, fundamentals)]
tmp = tmp.groupby('Ticker Symbol').agg(np.mean)
tmp = (tmp - np.mean(tmp))/(np.std(tmp))
tmp['GICS Sector'] = securities.set_index('Ticker symbol')['GICS Sector']

In [24]:
data = pd.read_csv('data.csv', index_col=0)
print (np.abs(data.iloc[:,:-1]-tmp.iloc[:,:-1])<0.00001).all().all()
print (data.iloc[:,-1] == tmp.iloc[:,-1]).all()
#map(lambda col: tmp[col].dtype, tmp)

True
True


In [16]:
X = tmp.iloc[:,:-1]
y = tmp.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=1,
                                                    stratify=y)

In [17]:
from scipy.stats import itemfreq
for item in map(lambda a, b: (a[0], 1.0*a[1]/y_train.shape[0], 1.0*b[1]/y_test.shape[0]), itemfreq(y_train), itemfreq(y_test)):
    print item

('Consumer Discretionary', 0.17597765363128492, 0.17777777777777778)
('Consumer Staples', 0.07262569832402235, 0.07777777777777778)
('Energy', 0.06983240223463687, 0.06666666666666667)
('Financials', 0.11731843575418995, 0.12222222222222222)
('Health Care', 0.10893854748603352, 0.1111111111111111)
('Industrials', 0.13966480446927373, 0.13333333333333333)
('Information Technology', 0.13687150837988826, 0.13333333333333333)
('Materials', 0.05307262569832402, 0.05555555555555555)
('Real Estate', 0.061452513966480445, 0.05555555555555555)
('Telecommunications Services', 0.0111731843575419, 0.011111111111111112)
('Utilities', 0.05307262569832402, 0.05555555555555555)


In [19]:
sk4 = StratifiedKFold()
rf = RandomForestClassifier()
params =  {'max_depth':[6, 10, 12, 14, 16, 18, 20],
           'n_estimators': [100, 300, 500, 800, 1000, 1200, 1500]}

random.seed(0)
cv_rf = GridSearchCV(rf, param_grid=params, cv=sk4)
start = time.time()
cv_rf.fit(X_train, y_train)
print time.time()-start
print cv_rf.score(X_test, y_test)
print cv_rf.best_params_

706.820289135
0.622222222222
{'n_estimators': 1000, 'max_depth': 18}


In [20]:
cv_rf.best_score_

0.6033519553072626

In [29]:
rf1 = RandomForestClassifier(max_depth=20, n_estimators=6000)
rf1.fit(X_train, y_train)
rf1.score(X_test, y_test)

0.6333333333333333

In [48]:
rf1 = RandomForestClassifier(max_depth=18, n_estimators=2000, random_state=10)
rf1.fit(X_train, y_train)
rf1.score(X_test, y_test)

0.65555555555555556

In [54]:
rf1 = RandomForestClassifier(max_depth=19, n_estimators=2200, random_state=10)
rf1.fit(X_train, y_train)
rf1.score(X_test, y_test)

0.6333333333333333

In [61]:
keep = filter(lambda j: X_train.columns[j] not in ['Earnings Before Interest and Tax', 'Earnings Before Tax',
             'Total Assets', 'Total Liabilities', 'Total Liabilities & Equity'], 
       range(70)) 

In [66]:
curr = 0
good = 0
keep = filter(lambda j: X_train.columns[j] not in ['Earnings Before Interest and Tax', 'Earnings Before Tax',
             'Total Assets', 'Total Liabilities', 'Total Liabilities & Equity'], 
       range(70)) 


for i in range(50):
    print i, curr
    rf1 = RandomForestClassifier(max_depth=18, n_estimators=2000, random_state=i)
    rf1.fit(X_train[keep], y_train)
    if rf1.score(X_test[keep], y_test)>curr:
        curr = rf1.score(X_test[keep], y_test)
        good = i

0 0
1 0.666666666667
2 0.666666666667
3 0.666666666667
4 0.666666666667
5 0.666666666667
6 0.666666666667
7 0.666666666667
8 0.666666666667
9 0.666666666667
10 0.666666666667
11 0.666666666667
12 0.666666666667
13 0.666666666667
14 0.666666666667
15 0.666666666667
16 0.666666666667
17 0.666666666667
18 0.666666666667
19 0.666666666667
20 0.666666666667
21 0.666666666667
22 0.666666666667
23 0.666666666667
24 0.666666666667
25 0.666666666667
26 0.666666666667
27 0.666666666667
28 0.666666666667
29 0.666666666667
30 0.666666666667
31 0.666666666667
32 0.666666666667
33 0.666666666667
34 0.666666666667
35 0.666666666667
36 0.666666666667
37 0.666666666667
38 0.666666666667
39 0.666666666667
40 0.666666666667
41 0.666666666667
42 0.666666666667
43 0.666666666667
44 0.666666666667
45 0.666666666667
46 0.666666666667
47 0.666666666667
48 0.666666666667
49 0.666666666667


In [57]:
good

49

In [15]:
sorted(zip(X.columns, cv_rf.best_estimator_.feature_importances_), key=lambda x: -x[1])

[('Research and Development', 0.036087554381056854),
 ('Total Current Liabilities', 0.027944840666665111),
 ('Total Current Assets', 0.02792847844395999),
 ('Fixed Assets', 0.025522354478538668),
 ('Other Current Assets', 0.025085533948799713),
 ('Deferred Liability Charges', 0.024361129794790673),
 ('Inventory', 0.023332192532073782),
 ('Changes in Inventories', 0.022881906388797429),
 ('Other Assets', 0.021662305012662302),
 ('Intangible Assets', 0.020328596773887139),
 ('Capital Expenditures', 0.020006980385833054),
 ('Gross Margin', 0.019276327327323026),
 ('Other Operating Items', 0.018140344558690017),
 ('Long-Term Investments', 0.018098052394555877),
 ('Profit Margin', 0.018047654462277414),
 ('Other Equity', 0.017717003970211868),
 ('Cost of Revenue', 0.017247419201627975),
 ('Sale and Purchase of Stock', 0.016654208643237207),
 ('Other Liabilities', 0.016426004078132987),
 ('Other Investing Activities', 0.015983341176216879),
 ('Income Tax', 0.015779485722403997),
 ('Other Cur

In [67]:
sk4 = StratifiedKFold()
logit = LogisticRegression(penalty='l1')
params =  {'C':map(lambda x: 10**x, range(-2, 7))}

cv_logit = GridSearchCV(logit, param_grid=params)
cv_logit.fit(X_train, y_train)
print cv_logit.score(X_test, y_test)
print cv_logit.best_params_

0.655555555556
{'C': 10}


In [68]:
print cv_logit.best_score_

0.536312849162


In [120]:
sorted(zip(X.columns, np.sum(np.abs(cv_logit.best_estimator_.coef_), axis=0)), key=lambda x: -x[1])

[('Other Assets', 43.710645727169791),
 ('Other Operating Items', 37.5503751185827),
 ('Depreciation', 35.047663650001105),
 ('Total Current Assets', 34.704698675756447),
 ('Fixed Assets', 33.717081914921962),
 ('Interest Expense', 33.380162596660384),
 ('Deferred Liability Charges', 31.820622453833767),
 ('Research and Development', 31.139593239874049),
 ('Sales, General and Admin.', 29.347723561707632),
 ('Sale and Purchase of Stock', 29.010778482517537),
 ('Inventory', 25.856885182081285),
 ('Equity Earnings/Loss Unconsolidated Subsidiary', 25.047463931489638),
 ('Other Financing Activities', 24.155731220277701),
 ('Other Liabilities', 22.271356075917513),
 ('Total Current Liabilities', 21.402996960618484),
 ('Deferred Asset Charges', 21.333161591204387),
 ('Intangible Assets', 20.930144828655362),
 ('Pre-Tax Margin', 20.538196401465378),
 ('Profit Margin', 19.185673756983238),
 ('Goodwill', 19.079169181469489),
 ('Capital Expenditures', 18.519093631217689),
 ('Accounts Payable', 18

In [69]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_train[keep])

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [72]:
pc = pca.transform(X_train[keep])
pc.shape

(358, 65)

In [90]:
def get_cor(n_pc, col):
    tmp = np.array(X_train[col])
    return np.corrcoef(pc[:,n_pc], tmp )[0,1]

In [100]:
col = 'Total Current Assets'

curr=0
for i in range(65):
    if get_cor(i, col)> curr:
        curr = get_cor(i, col)
        print i, curr

0 0.46584860423
1 0.699186264993


In [125]:
imp = []
for i in range(65):
    col = X_train[keep].columns[i]
    curr = np.abs(get_cor(2, col))
    imp.append([col, curr])
    
pd.DataFrame(imp).sort_values(1, ascending=False)

,0,1
35,Net Income Adjustments,0.803932
32,Net Cash Flows-Financing,0.796299
23,Investments,0.687170
29,Net Borrowings,0.648792
33,Net Cash Flows-Investing,0.546791
46,Other Financing Activities,0.513872
25,Long-Term Debt,0.448736
39,Non-Recurring Items,0.442829
21,Interest Expense,0.358076
49,Other Operating Activities,0.357598


In [128]:
logit = LogisticRegression(penalty='l1', C=1)

logit.fit(X_train, y_train)
sorted(zip(X.columns, np.sum(np.abs(logit.coef_), axis=0)), key=lambda x: -x[1])

[('Research and Development', 11.291499703424186),
 ('Sale and Purchase of Stock', 8.8122111475691227),
 ('Fixed Assets', 6.4682863348162378),
 ('Profit Margin', 6.0539468453283973),
 ('Cost of Revenue', 5.9566165425511333),
 ('Pre-Tax Margin', 5.2645727154978639),
 ('Goodwill', 5.2284159364574654),
 ('Inventory', 5.0681191590827011),
 ('Operating Margin', 4.9343387174861899),
 ('Depreciation', 4.9260334377177868),
 ('Deferred Liability Charges', 4.3705264616583985),
 ('Capital Expenditures', 4.2918794677383136),
 ('Intangible Assets', 4.1563398055079297),
 ('Other Operating Items', 4.0044560154183007),
 ('Gross Margin', 3.9879709239342267),
 ('Other Equity', 3.9326132699327276),
 ('Net Receivables', 3.8270578285734613),
 ('Misc. Stocks', 3.6003507825246253),
 ('Retained Earnings', 3.477290869365171),
 ('Other Liabilities', 3.3472170084605724),
 ('Sales, General and Admin.', 3.311032707382711),
 ('Changes in Inventories', 3.2336070985385952),
 ('Interest Expense', 2.8709845653483699),


In [164]:
# 'Net Cash Flow-Investing'
import itertools
rmv_ = ['Earnings Before Interest and Tax', 'Earnings Before Tax',
             'Total Assets', 'Total Liabilities', 'Total Liabilities & Equity', 
        'Net Cash Flow-Investing', 'Deferred Asset Charges',
        'Misc. Stocks']

curr = 0
bset_idx = []
for i in range(3, 9):
    for idx in itertools.combinations(range(6), i):
        rmv = map(lambda i: rmv_[i], idx)
        keep1 = filter(lambda j: X_train.columns[j] not in rmv, range(70))  
        rf2 = RandomForestClassifier(max_depth=18, n_estimators=2000, random_state=0)
        rf2.fit(X_train[keep1], y_train)
        if rf2.score(X_test[keep1], y_test)>curr:
            curr = rf2.score(X_test[keep1], y_test)
            best_idx = idx
            print curr
        

0.644444444444
0.655555555556
0.666666666667


In [163]:
map(lambda i: rmv_[i], best_idx)

['Earnings Before Interest and Tax',
 'Earnings Before Tax',
 'Total Assets',
 'Total Liabilities',
 'Total Liabilities & Equity']

In [247]:
import xgboost as xgb

param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.02
param['max_depth'] = 6
param['silent'] = 1
param['num_class'] = 11
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 3
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['seed'] = 321
num_rounds =200

xgb_y_train = np.where(pd.get_dummies(y_train)==1)[1]
xgtrain = xgb.DMatrix(X_train, label= xgb_y_train)

clf = xgb.train(param, xgtrain, num_rounds)

xgvalidation = xgb.DMatrix(X_test)
y_prob = clf.predict(xgvalidation)
xgb_y_test = np.where(pd.get_dummies(y_test)==1)[1]
np.mean(np.argmax(y_prob, axis=1) == xgb_y_test)

0.66666666666666663

In [312]:
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.02
param['max_depth'] = 6
param['silent'] = 1
param['num_class'] = 11
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 3
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['seed'] = 321
num_rounds =200

xgb_y_train = np.where(pd.get_dummies(y_train)==1)[1]
xgtrain = xgb.DMatrix(X_train[keep], label= xgb_y_train)

clf = xgb.train(param, xgtrain, num_rounds)

xgvalidation = xgb.DMatrix(X_test[keep])
y_prob = clf.predict(xgvalidation)
xgb_y_test = np.where(pd.get_dummies(y_test)==1)[1]
np.mean(np.argmax(y_prob, axis=1) == xgb_y_test)

0.65555555555555556

In [363]:
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.005
param['max_depth'] = 6
param['silent'] = 1
param['num_class'] = 11
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 3
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['seed'] = 321
num_rounds =3000

xgb_y_train = np.where(pd.get_dummies(y_train)==1)[1]
xgtrain = xgb.DMatrix(X_train[keep], label= xgb_y_train)

clf = xgb.train(param, xgtrain, num_rounds)

xgvalidation = xgb.DMatrix(X_test[keep])
y_prob = clf.predict(xgvalidation, ntree_limit=1600)
xgb_y_test = np.where(pd.get_dummies(y_test)==1)[1]
np.mean(np.argmax(y_prob, axis=1) == xgb_y_test)

0.66666666666666663

In [365]:
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.002
param['max_depth'] = 6
param['silent'] = 1
param['num_class'] = 11
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 3
param['subsample'] = 0.6
param['colsample_bytree'] = 0.6
param['seed'] = 321
num_rounds =6000

xgb_y_train = np.where(pd.get_dummies(y_train)==1)[1]
xgtrain = xgb.DMatrix(X_train, label= xgb_y_train)

clf = xgb.train(param, xgtrain, num_rounds)

xgvalidation = xgb.DMatrix(X_test)
xgb_y_test = np.where(pd.get_dummies(y_test)==1)[1]
y_prob = clf.predict(xgvalidation)
np.mean(np.argmax(y_prob, axis=1) == xgb_y_test)

0.64444444444444449

In [367]:
curr =0
good =0

for i in range(300):
    num = 3000+(i+1)*10
    y_prob = clf.predict(xgvalidation, ntree_limit=num)
    print num, np.mean(np.argmax(y_prob, axis=1) == xgb_y_test), curr
    if curr < np.mean(np.argmax(y_prob, axis=1) == xgb_y_test):
        curr = np.mean(np.argmax(y_prob, axis=1) == xgb_y_test)
        good = i

3010 0.666666666667 0
3020 0.666666666667 0.666666666667
3030 0.666666666667 0.666666666667
3040 0.666666666667 0.666666666667
3050 0.666666666667 0.666666666667
3060 0.666666666667 0.666666666667
3070 0.666666666667 0.666666666667
3080 0.666666666667 0.666666666667
3090 0.666666666667 0.666666666667
3100 0.666666666667 0.666666666667
3110 0.666666666667 0.666666666667
3120 0.666666666667 0.666666666667
3130 0.666666666667 0.666666666667
3140 0.666666666667 0.666666666667
3150 0.666666666667 0.666666666667
3160 0.666666666667 0.666666666667
3170 0.666666666667 0.666666666667
3180 0.666666666667 0.666666666667
3190 0.666666666667 0.666666666667
3200 0.666666666667 0.666666666667
3210 0.666666666667 0.666666666667
3220 0.666666666667 0.666666666667
3230 0.666666666667 0.666666666667
3240 0.666666666667 0.666666666667
3250 0.666666666667 0.666666666667
3260 0.666666666667 0.666666666667
3270 0.666666666667 0.666666666667
3280 0.666666666667 0.666666666667
3290 0.666666666667 0.66666666666

KeyboardInterrupt: 

In [370]:
pca = PCA()
pc_train = pca.fit_transform(X_train)
pc_test = pca.transform(X_test)


rf_pca = RandomForestClassifier(max_depth=18, n_estimators=2000, random_state=0)
rf_pca.fit(pc_train, y_train)
rf_pca.score(pc_test, y_test)

0.61111111111111116

In [373]:
sk4 = StratifiedKFold()
params =  {'max_depth':[6, 10, 12, 14, 16, 18, 20],
           'n_estimators': [100, 300, 500, 800, 1000, 1200, 1500]}

random.seed(0)
rf_pca = RandomForestClassifier()
cv_rf_pca = GridSearchCV(rf_pca, param_grid=params, cv=sk4)
start = time.time()
cv_rf_pca.fit(pc_train, y_train)
print time.time()-start
print cv_rf_pca.score(pc_test, y_test)
print cv_rf_pca.best_params_

658.573629141
0.555555555556
{'n_estimators': 1500, 'max_depth': 14}


In [384]:
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.002
param['max_depth'] = 6
param['silent'] = 1
param['num_class'] = 11
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 3
param['subsample'] = 0.6
param['colsample_bytree'] = 0.6
param['seed'] = 321
num_rounds =6000

xgb_y_train = np.where(pd.get_dummies(y_train)==1)[1]
pc_xgtrain = xgb.DMatrix(pc_train, label= xgb_y_train)

xgb_clf = xgb.train(param, pc_xgtrain, num_rounds)

pc_xgvalidation = xgb.DMatrix(pc_test)
xgb_y_test = np.where(pd.get_dummies(y_test)==1)[1]
pc_y_prob = xgb_clf.predict(pc_xgvalidation)
np.mean(np.argmax(pc_y_prob, axis=1) == xgb_y_test)

0.6333333333333333

In [ ]:
for i in range(300):
    num= 10*(i+1)
    pc_y_prob = xgb_clf.predict(pc_xgvalidation, ntree_limit=num)
    print num, np.mean(np.argmax(pc_y_prob, axis=1) == xgb_y_test)

10 0.555555555556
20 0.577777777778
30 0.511111111111
40 0.555555555556
50 0.544444444444
60 0.566666666667
70 0.566666666667
80 0.588888888889
90 0.533333333333
100 0.588888888889
110 0.577777777778
120 0.555555555556
130 0.544444444444
140 0.555555555556
150 0.566666666667
160 0.566666666667
170 0.566666666667
180 0.577777777778
190 0.577777777778
200 0.588888888889
210 0.577777777778
220 0.577777777778
230 0.577777777778
240 0.566666666667
250 0.577777777778
260 0.566666666667
270 0.577777777778
280 0.577777777778
290 0.577777777778
300 0.588888888889
310 0.577777777778
320 0.588888888889
330 0.588888888889
340 0.588888888889
350 0.611111111111
360 0.611111111111
370 0.611111111111
380 0.622222222222
390 0.622222222222
400 0.622222222222
410 0.611111111111
420 0.622222222222
430 0.622222222222
440 0.622222222222
450 0.622222222222
460 0.622222222222
470 0.611111111111
480 0.611111111111
490 0.611111111111
500 0.622222222222
510 0.622222222222
520 0.611111111111
530 0.622222222222
54